In [2]:
import requests
import pandas as pd
import json
import ast
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [6]:
file_path="../Project 1/movies_metadata.csv"
movies_df=pd.read_csv(file_path)

#cr_file_path="Resources/credits.csv"
#credits_df=pd.read_csv(cr_file_path)

#ratings_file_path="Resources/ratings.csv"
#ratings_df=pd.read_csv(ratings_file_path)


C:\Users\iffym\AppData\Local\Temp\ipykernel_18136\2776802373.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df=pd.read_csv(file_path)
